# Dutch UMLS to concept table
This notebook describes how to convert a UMLS concept table containing Dutch terms, to a formatted concept table to be used in a tool such as MedCAT. In the second part of this notebook, we add drug names from Dutch SNOMED, because these concepts are not well represented in the Dutch UMLS source vocabularies. A large scale automatic mapping from SNOMED Dutch to UMLS is not possible because there of many-to-mapping mapping.

Requirements:
- MySQL database containing Dutch UMLS terms

For adding Dutch SNOMED drug names:
- Dutch SNOMED concept tablel, created in `dutch-snomed_to_concept-table.ipynb`
- MySQL database containing SNOMED-US, which is used for mapping SNOMED Dutch -> UMLS

In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json
import re
import os

In [2]:
# Credentials to connect to UMLS MySQL database
load_dotenv()
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
database = os.getenv('MYSQL_DATABASE')

# Create the connection
connection_string = f'mysql://{user}:{password}@{host}:{port}/{database}'
connection = create_engine(connection_string)

In [3]:
# Retrieve Dutch UMLS concepts
query = """
SELECT * FROM MRCONSO WHERE LAT = 'DUT'
"""
df_dutch_umls = pd.read_sql_query(query, con=connection)
df_dutch_umls.head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0030271,DUT,S,L2070920,PF,S2409142,N,A10134406,None,None,10065249,MDRDUT,PT,10065249,Pancoast-syndroom,3,N,None
1,C0238106,DUT,P,L4955608,PF,S5643112,Y,A10134408,None,None,10009657,MDRDUT,PT,10009657,Clostridium difficile-colitis,3,N,None
2,C0851107,DUT,P,L4955853,PF,S5643358,Y,A10134409,None,None,10050681,MDRDUT,PT,10050681,Epstein-Barr-virustest,3,N,None
3,C0035232,DUT,S,L4958848,PF,S5646353,Y,A10134412,None,None,10038708,MDRDUT,PT,10038708,ademhalingsverlamming,3,N,None
4,C0400161,DUT,P,L4959926,PF,S5647430,Y,A10134414,None,None,10002169,MDRDUT,PT,10002169,anale poliepectomie,3,N,None


## Term type in source
Some source-defined term types are not relevant for our use case. In the next part we will drop those. See https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html 

In [4]:
df_dutch_umls.TTY.value_counts()

PT     112383
LLT     73284
LN      54641
MH      28618
SY      11859
HT       3296
HG        337
SMQ       228
CP         38
OS         27
AB         27
Name: TTY, dtype: int64

| TTY  | Description | Count | Example | Reference|
| - | - | - | - | - |
| PT | Designated preferred name| 111766 | harthypertrofie, Pancoast-syndroom ||
| LLT | Lower Level Term | 71603 | heupkombreuk, buikkramp| |
| LN | LOINC official fully specified name | 52313 | fencyclidine:massa/massa:moment:haar:kwantitatief | |
| MH | Main heading | 28657 | Dehydratie, Astma | |
| SY | Designated synonym | 11863 | Spanningshoofdpijn, Ziekte van Hodgkin | |
| OL | Non-current Lower Level Term| 9291 | acquired immunodeficiency syndrome, ankylose van gewricht, meerdere plaatsen | https://meddra.org/sites/default/files/page/documents_insert/meddra_-_terminologies_coding.pdf |
| HT | Hierarchical term | 3295 | calciummetabolismestoornissen, oculaire hemorragische aandoeningen	 | |
| LO | Obsolete official fully specified name | 1696| promyelocyten/100 leukocyten:getalsfractie:mom...	| |
| HG | High Level Group Term |  337| complicaties geassocieerd met medisch hulpmiddel, zuur-basestoornissen | |
| SMQ| Standardised MedDRA Query |  225| Leveraandoeningen (SMQ) , Tumormarkers (SMQ) | |
| CP | ICPC component process (in original form) |   38| Ander bloedonderzoek, Medicatie/recept/injectie | |
| OS | System-organ class |   27| Bloed- en lymfestelselaandoeningen, Infecties en parasitaire aandoeningen | |
| AB | Abbreviation in any source vocabulary |   27| Infec, Neopl, Ear, Endo | |

In [5]:
# Select a set of TTYs that seem most relevant for entity linking
tty_selection = ['PT', 'LLT', 'MH', 'SY']
df_dutch_umls_subset = df_dutch_umls[df_dutch_umls.TTY.isin(tty_selection)].copy()

# Keep only relevant columns
df_dutch_umls_subset = df_dutch_umls_subset[['CUI', 'STR', 'TTY', 'SAB']]
df_dutch_umls_subset.rename({'CUI': 'cui', 'STR': 'str', 'TTY': 'tty', 'SAB': 'sab'}, inplace=True, axis=1)

# Most of the terms in UMLS have the Metathesaurus preferred name in English. 
# For a rough but effective fix to get a good preferred name for the Dutch terms, 
# change the terms that have the value "Designated preferred name" (PT) for the 
# Term Type in Source (TTY) to the Metathesaurus preferred name (PN). All others
# can be saved as synonym.
df_dutch_umls_subset.tty.replace({'PT': 'PN',
                                  'LLT': 'SY',
                                  'MH': 'SY'}, inplace=True)

# Remove "NAO" ("Niet Anders Omschreven"), which is relevant for the source terminlogy but not for entity linking.
# See https://meddra.org/sites/default/files/guidance/file/intguide_15_0_dutch.pdf
df_dutch_umls_subset.str = df_dutch_umls_subset.str.replace({' NAO': '', ' \(NAO\)': '', ' nao': ''}, regex=True)

# Sort values
df_dutch_umls_subset.sort_values(by=['cui', 'tty', 'str', 'sab'], inplace=True)

# Drop duplicates, only keep the first entry (which is a PN because we sorted)
print(f'Records before dropping duplicates: {df_dutch_umls_subset.shape[0]}')
df_dutch_umls_subset = df_dutch_umls_subset.drop_duplicates(subset=['cui', 'str'], keep='first').reset_index(drop=True)
print(f'Records after dropping duplicates: {df_dutch_umls_subset.shape[0]}')

# Because dropped duplicates, only the first value in SAB is saved. Because we lost the information of other colums, rename the values to UMLS-dutch
df_dutch_umls_subset['sab'] = 'UMLS-dutch'
df_dutch_umls_subset.head(20)

Records before dropping duplicates: 226144
Records after dropping duplicates: 187795


,cui,str,tty,sab
0,C0000696,A-zenuwvezels,SY,UMLS-dutch
1,C0000715,Abattoir,SY,UMLS-dutch
2,C0000715,Abattoirs,SY,UMLS-dutch
3,C0000722,Abbreviated Injury Scale,SY,UMLS-dutch
4,C0000726,Abdomen,SY,UMLS-dutch
5,C0000726,Buik,SY,UMLS-dutch
6,C0000727,Acute buik,PN,UMLS-dutch
7,C0000727,abdomen; acute buik,PN,UMLS-dutch
8,C0000727,"abdominaal; syndroom, acuut",PN,UMLS-dutch
9,C0000727,acuut abdomen,PN,UMLS-dutch


# Add SNOMED

### Load SNOMED US

In [6]:
query = "SELECT distinct cui, scui FROM MRCONSO where sab = 'SNOMEDCT_US'"
df_snomed_us = pd.read_sql_query(query, con=connection)
df_snomed_us.scui = df_snomed_us.scui.astype(str)
print(f'SNOMED US terms with UMLS CUI: {df_snomed_us.shape[0]}')
df_snomed_us.head()

SNOMED US terms with UMLS CUI: 363794


,cui,scui
0,C0000052,58488005
1,C0000097,285407008
2,C0000102,13579002
3,C0000163,112116001
4,C0000167,46120009


In [7]:
snomed_to_umls_mapping = df_snomed_us.groupby('scui')['cui'].apply(list).to_dict()
print(f'Number of SNOMED IDs that map to at least 1 CUI: {len(snomed_to_umls_mapping)}')

Number of SNOMED IDs that map to at least 1 CUI: 361461


In [8]:
unambiguous_mapping_ids = set()
for snomed_id in snomed_to_umls_mapping:
    if len(snomed_to_umls_mapping[snomed_id]) == 1:
        unambiguous_mapping_ids.add(snomed_id)
print(len(unambiguous_mapping_ids))
print(f'Number of SNOMED IDs that map to only 1 CUI: {len(unambiguous_mapping_ids)}')

359388
Number of SNOMED IDs that map to only 1 CUI: 359388


## Load SNOMED NL

In [9]:
df_snomed_dutch = pd.read_csv('04_ConceptDB/snomedct-dutch_v1.0.csv', dtype=str)
df_snomed_dutch.head()

,cui,str,tty,tui,sab
0,104001,excisie van afwijkend weefsel van patella,PN,verrichting,SNOMED-CT-NL
1,104001,excisie van laesie van knieschijf,SY,verrichting,SNOMED-CT-NL
2,106004,structuur van posterieure carpale regio,PN,lichaamsstructuur,SNOMED-CT-NL
3,106004,posterieur gebied van handwortel,SY,lichaamsstructuur,SNOMED-CT-NL
4,106004,posterieur carpaal gebied,SY,lichaamsstructuur,SNOMED-CT-NL


In [10]:
df_snomed_dutch.shape

(413790, 5)

In [11]:
df_dutch_umls_subset.head()

,cui,str,tty,sab
0,C0000696,A-zenuwvezels,SY,UMLS-dutch
1,C0000715,Abattoir,SY,UMLS-dutch
2,C0000715,Abattoirs,SY,UMLS-dutch
3,C0000722,Abbreviated Injury Scale,SY,UMLS-dutch
4,C0000726,Abdomen,SY,UMLS-dutch


## Add SNOMED NL to UMLS

In [12]:
dutch_umls_ids=df_dutch_umls_subset.groupby('cui')['str'].apply(list).to_dict()

dutch_umls_names_lowercase = set()
for cui in dutch_umls_ids:
    for value in dutch_umls_ids[cui]:
        dutch_umls_names_lowercase.add(value.lower())

In [13]:
df_snomed_dutch['lowercase_str'] = df_snomed_dutch.str.str.lower()

In [14]:
def map_dutch_snomed_to_umls(row):
    snomed_id = row['cui']
    if snomed_id in unambiguous_mapping_ids:
        cui = snomed_to_umls_mapping[snomed_id][0]
        
        # Check whether SNOMED name is a name in UMLS, under any CUI.
        # This is to prevent:
        # - Adding names for a concept that we already have.
        # - Introducing concepts that are already in our DB but map to a different CUI
        #   because of one-to-many SNOMED to UMLS mapping. 
        # The downside is that concepts that are not in our DB yet, will not be added. 
        if row['lowercase_str'] not in dutch_umls_names_lowercase:
            
            # Check if the term is new, or already exists and therefor always is a synonym.
            if cui in dutch_umls_ids:
                snomed_names_to_add.append([cui, row['str'], 'SY'])
            else:
                snomed_names_to_add.append([cui, row['str'], row['tty']])
            
snomed_names_to_add = list()

# Apply function
df_snomed_dutch.apply(map_dutch_snomed_to_umls, axis = 1)

print(len(snomed_names_to_add))

378808


In [15]:
snomed_names_with_cui = pd.DataFrame(snomed_names_to_add, columns = ['cui', 'str', 'tty'])
snomed_names_with_cui['sab'] = 'SNOMEDCT-NL'
snomed_names_with_cui.head()

,cui,str,tty,sab
0,C0187893,excisie van afwijkend weefsel van patella,PN,SNOMEDCT-NL
1,C0187893,excisie van laesie van knieschijf,SY,SNOMEDCT-NL
2,C0230364,structuur van posterieure carpale regio,PN,SNOMEDCT-NL
3,C0230364,posterieur gebied van handwortel,SY,SNOMEDCT-NL
4,C0230364,posterieur carpaal gebied,SY,SNOMEDCT-NL


In [16]:
df_dutch_umls_subset.shape

(187795, 4)

In [17]:
snomed_names_with_cui.shape

(378808, 4)

In [18]:
umls_snomed_merged = pd.concat([df_dutch_umls_subset, snomed_names_with_cui])
umls_snomed_merged.shape

(566603, 4)

In [19]:
umls_snomed_merged.sort_values(by=['cui', 'tty', 'sab', 'str'], inplace=True)
umls_snomed_merged.reset_index(drop=True,inplace=True)

## Remove problematic names


In [20]:
names_to_remove = ['Bij', # C0004923
                   'Bijen', # C0004923
                   'Haar', # C0018494
                   'bleek', # C0678215
                   'Weer', # C0043085
                   'Na+'] # C0337443
umls_snomed_merged[umls_snomed_merged.str.isin(names_to_remove)]

,cui,str,tty,sab
4528,C0004923,Bij,SY,UMLS-dutch
4529,C0004923,Bijen,SY,UMLS-dutch
20063,C0018494,Haar,SY,UMLS-dutch
49866,C0043085,Weer,SY,UMLS-dutch
170041,C0337443,Na+,SY,UMLS-dutch
308726,C0678215,bleek,SY,UMLS-dutch


In [21]:
# Remove rows
rows_to_remove = umls_snomed_merged[umls_snomed_merged.str.isin(names_to_remove)].index
print(f'Number of rows before removing rows: {umls_snomed_merged.shape[0]}')
umls_snomed_filtered = umls_snomed_merged.drop(umls_snomed_merged.index[rows_to_remove])
print(f'Number of rows before removing rows: {umls_snomed_filtered.shape[0]}')

Number of rows before removing rows: 566603
Number of rows before removing rows: 566597


## Add custom CUIs
Sometimes names or concept are not captured in any of the Dutch terminologies. By looking up the English name for these concepts, we can add custom Dutch names using the real UMLS identifier.

In [22]:
umls_snomed_filtered.head()

,cui,str,tty,sab
0,C0000097,methyl-fenyltetrahydropyridine,PN,SNOMEDCT-NL
1,C0000097,"1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine",SY,SNOMEDCT-NL
2,C0000097,MPTP,SY,SNOMEDCT-NL
3,C0000215,"2,4,5-trichloorfenoxyazijnzuur",PN,SNOMEDCT-NL
4,C0000215,"2,4,5-T",SY,SNOMEDCT-NL


In [23]:
custom_concepts = pd.read_csv("custom_concepts.csv")
custom_concepts

,cui,str,tty,sab
0,C0456984,uitslag,SY,UMCU
1,C0019080,bloedt,SY,UMCU
2,C0019080,bloeden,SY,UMCU
3,C0225844,RA,SY,UMCU
4,C0225883,RV,SY,UMCU
5,C0225897,LV,SY,UMCU


In [24]:
print(f'Number of rows before adding rows: {umls_snomed_filtered.shape[0]}')
umls_snomed_custom = pd.concat([umls_snomed_filtered, custom_concepts])
print(f'Number of rows after adding rows: {umls_snomed_custom.shape[0]}')

Number of rows before adding rows: 566597
Number of rows after adding rows: 566603


## Add TUI (types)
UMLS concepts have one or multiple types. These types are kept in a separate table, `MRSTY`. See https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt for all types.

In [25]:
# Load TUI table from MySQL
query = """
SELECT cui, tui, sty FROM MRSTY
"""
df_tui = pd.read_sql_query(query, con=connection)

In [26]:
# Add TUI column to previously created dataframe
umls_snomed_tui = umls_snomed_custom.merge(df_tui, how='left', on='cui')

# View some concepts that have multiple TUIs
umls_snomed_tui[umls_snomed_tui.duplicated(subset=['cui', 'str'], keep=False)].head(10)

,cui,str,tty,sab,tui,sty
0,C0000097,methyl-fenyltetrahydropyridine,PN,SNOMEDCT-NL,T131,Hazardous or Poisonous Substance
1,C0000097,methyl-fenyltetrahydropyridine,PN,SNOMEDCT-NL,T109,Organic Chemical
2,C0000097,"1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine",SY,SNOMEDCT-NL,T131,Hazardous or Poisonous Substance
3,C0000097,"1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine",SY,SNOMEDCT-NL,T109,Organic Chemical
4,C0000097,MPTP,SY,SNOMEDCT-NL,T131,Hazardous or Poisonous Substance
5,C0000097,MPTP,SY,SNOMEDCT-NL,T109,Organic Chemical
6,C0000215,"2,4,5-trichloorfenoxyazijnzuur",PN,SNOMEDCT-NL,T131,Hazardous or Poisonous Substance
7,C0000215,"2,4,5-trichloorfenoxyazijnzuur",PN,SNOMEDCT-NL,T109,Organic Chemical
8,C0000215,"2,4,5-T",SY,SNOMEDCT-NL,T131,Hazardous or Poisonous Substance
9,C0000215,"2,4,5-T",SY,SNOMEDCT-NL,T109,Organic Chemical


In [27]:
print(f'Number of unique TUIs in Dutch UMLS subset: {len(umls_snomed_tui.tui.unique())}')

Number of unique TUIs in Dutch UMLS subset: 125


In [28]:
# Create dataframe with counts per TUI name
type_counts = umls_snomed_tui.sty.value_counts().to_frame()
type_counts_tui = umls_snomed_tui.sty.value_counts().to_frame()
type_counts.reset_index(inplace=True)

# Add TUI code
type_counts_tui = umls_snomed_tui.tui.value_counts().to_frame()
tuis = type_counts_tui.index
type_counts['tui'] = tuis

# Format nicely
type_counts.columns = ['sty', 'count', 'tui']
type_counts = type_counts[['tui', 'sty', 'count']]
type_counts

,tui,sty,count
0,T047,Disease or Syndrome,128883
1,T061,Therapeutic or Preventive Procedure,75926
2,T023,"Body Part, Organ, or Organ Component",63297
3,T033,Finding,49985
4,T037,Injury or Poisoning,42631
...,...,...,...
120,T171,Carbohydrate Sequence,4
121,T088,Language,4
122,T021,Fully Formed Anatomical Structure,3
123,T103,Chemical,2


## TUI Filtering
We could implement filtering of TUIs here. This depends on the domain and question of subsequent analysis. For SNOMED

In [29]:
tuis_to_remove = ['T078', # Idea or Concept
                 ]
umls_snomed_tui[umls_snomed_tui.tui.isin(tuis_to_remove)].head()

,cui,str,tty,sab,tui,sty
2607,C0003057,"Dier, rechten van het",SY,UMLS-dutch,T078,Idea or Concept
4815,C0004942,Behaviorisme,SY,UMLS-dutch,T078,Idea or Concept
4845,C0004978,Weldadigheid,SY,UMLS-dutch,T078,Idea or Concept
5132,C0005488,"Kwesties, bioethische",SY,UMLS-dutch,T078,Idea or Concept
6481,C0006343,Boeddhisme,SY,UMLS-dutch,T078,Idea or Concept


In [30]:
# Remove rows
rows_to_remove = umls_snomed_tui[umls_snomed_tui.tui.isin(tuis_to_remove)].index
print(f'Number of rows before removing rows: {umls_snomed_tui.shape[0]}')
umls_snomed_tui_filtered = umls_snomed_tui.drop(umls_snomed_tui.index[rows_to_remove])
print(f'Number of rows before removing rows: {umls_snomed_tui_filtered.shape[0]}')

Number of rows before removing rows: 574201
Number of rows before removing rows: 573886


## Column Names
In MedCAT v1.0 the column name specification has changed and is defined as in the [README.md in examples](https://github.com/CogStack/MedCAT/tree/master/examples).

In [31]:
umls_snomed_tui_filtered.rename(columns={'str': 'name', 'tty': 'name_status', 'sab': 'ontologies', 'tui': 'type_ids'}, inplace=True)
umls_snomed_tui_filtered.drop(['sty'], axis = 1, inplace=True)
umls_snomed_tui_filtered.head()

,cui,name,name_status,ontologies,type_ids
0,C0000097,methyl-fenyltetrahydropyridine,PN,SNOMEDCT-NL,T131
1,C0000097,methyl-fenyltetrahydropyridine,PN,SNOMEDCT-NL,T109
2,C0000097,"1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine",SY,SNOMEDCT-NL,T131
3,C0000097,"1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine",SY,SNOMEDCT-NL,T109
4,C0000097,MPTP,SY,SNOMEDCT-NL,T131


In [32]:
umls_snomed_tui_filtered.shape

(573886, 5)

## Saving

In [33]:
umls_snomed_tui_filtered.to_csv('04_ConceptDB/umls-dutch_v1.7.csv', index=False)
type_counts.to_csv('04_ConceptDB/tuis-umls-dutch_v1.7.csv', index = False, sep='\t')